**Solution for migrating all user data between ArcGIS Online/Enterprise Orgs.**

Requirements:
 - Administrator level permissions on the "Source Org"

Required Steps:
 - add source org credentials to "credentials/credentials_source.json"
 - add target org credentials to "credentials/credentials_target.json"

Enjoy :)

In [1]:
from arcgis.gis import GIS, Item
from arcgis.env import active_gis
from arcgis.features import FeatureLayerCollection
from arcgis.mapping import WebMap
from common_lib import get_user_items, print_user_inventory
from pathlib import Path
import sys

try:
    from ujson import loads
except ModuleNotFoundError:
    from json import loads


def authenticate_gis(in_json, verify_cert=False):
    # load source credentials
    f = open(in_json, "r")
    creds = loads(f.read())
    # Closing file
    f.close()

    # Authenticate
    return GIS(creds['portal'], creds['username'], creds['password'], verify_cert=verify_cert)

source = authenticate_gis('credentials/credentials_source.json', verify_cert=False)
print("Source Org: {0}".format(source))
print("Source Access User: {0} \n".format(source.users.me))

target = authenticate_gis('credentials/credentials_target.json', verify_cert=False)
print("Target Org: {0}".format(target))
print("Target Access User: {0}".format(target.users.me))

print("\n **Proceed only if the above information is correct**")

Source Org: GIS @ https://CSOGRD.maps.arcgis.com version:9.1
Source Access User: <User username:gtaylor_CSOGRD> 

Target Org: GIS @ https://IVT.maps.arcgis.com version:9.1
Target Access User: <User username:gtaylor_IVT>

 **Proceed only if the above information is correct**


In [2]:
# Clone Content from all users

users = source.users.search(max_users=1000)
print("Detected Source Org Users for Copying items from as: \n")
for n in [user.username for user in users]:
    print(n)
print("\n **Proceed only if the above information is correct**")

Detected Source Org Users for Copying items from as: 

acrawford_CSOGRD
AkoshuaA_CSOGRD
andre.maynard93_CSOGRD1
AnichellT_CSOGRD
CannishaB_CSOGRD
DexterC_CSOGRD
EdricaC_CSOGRD
EthronS_CSOGRD
GillianB_CSOGRD
gtaylor_CSOGRD
JeddM_CSOGRD
jmccune_CSOGRD
JoanR_CSOGRD
jsill_CSOGRD
kam_CSOGRD
kbeem_CSOGRD
khess_CSOGRD
klogie_CSOGRD
KristelL_CSOGRD
ksmyth_CSOGRD
LisaA_CSOGRD
LouraineM_CSOGRD
MichelleF_CSOGRD
mitchell.jerry_CSOGRD
NikkiR_CSOGRD
NorjahS_CSOGRD
paulorumen_CSOGRD
SherniaJ_CSOGRD
Shimronmurray_CSOGRD

 **Proceed only if the above information is correct**


In [6]:
bypass_items_list = ["Points From Inbox"]

# Iteration 1 for base features & Service Definitions
for user in users:
    username = user.username
    # TODO: Get list of all user items.
    user_inventory = source.content.search("owner:{0}".format(username))
    if user_inventory:
        for i in user_inventory:
            print(i.type)
            # Note: Feature Layer Collections appears as Feature Service on i.type
            if i.type not in ["Web Map", "Web Scene", "Form", "Web Mapping Application"] and "type:Feature Layer Collection" not in str(i):
                if i.title not in bypass_items_list:
                    # Ensure item does not exist on target org
                    if len(target.content.search("title:{0}".format(i.title), item_type=i.type, max_items=1000)) > 0:
                        print("detected item {0} already on target machine... skipping".format(i))
                    else: # Upload item to target org
                        print("transferring {0}".format(i))
                        cloned_items = target.content.clone_items(items=[i])
'''
# Iteration 2 for Feature Layer Collection
for user in users:
    username = user.username
    # TODO: Get list of all user items.
    user_inventory = source.content.search("owner:{0}".format(username))
    if user_inventory:
        for i in user_inventory:
            print(i.type)
            # Note: Feature Layer Collections appears as Feature Service on i.type
            if "type:Feature Layer Collection" in str(i):
                # Ensure item does not exist on target org
                if len(target.content.search("title:{0}".format(i.title), item_type=i.type, max_items=1000)) > 0:
                    print("detected item {0} already on target machine... skipping".format(i))
                else: # Upload item to target org
                    print("transferring {0}".format(i))
                    cloned_items = target.content.clone_items(items=[i])
'''
# Iteration 3 for Maps and Scenes
for user in users:
    username = user.username
    # TODO: Get list of all user items.
    user_inventory = source.content.search("owner:{0}".format(username))
    if user_inventory:
        for i in user_inventory:
            print(i.type)
            # Note: Feature Layer Collections appears as Feature Service on i.type
            if i.type in ["Web Map", "Web Scene"] and "type:Feature Layer Collection" not in str(i):
                if i.title not in bypass_items_list:
                    # Ensure item does not exist on target org
                    if len(target.content.search("title:{0}".format(i.title), item_type=i.type, max_items=1000)) > 0:
                        print("detected item {0} already on target machine... skipping".format(i))
                    else: # Upload item to target org
                        print("transferring {0}".format(i))
                        cloned_items = target.content.clone_items(items=[i])


# Iteration 4 for all other Apps

Scene Package
detected item <Item title:"testSingleDeadTree" type:Scene Package owner:acrawford_CSOGRD> already on target machine... skipping
Scene Package
detected item <Item title:"Grenada 3D Buildings not segmented" type:Scene Package owner:acrawford_CSOGRD> already on target machine... skipping
Web Map
Scene Package
detected item <Item title:"Grenada_PCSLKP_Bldg_Colorized" type:Scene Package owner:acrawford_CSOGRD> already on target machine... skipping
Feature Service
Scene Package
detected item <Item title:"Grenada Colored Roof Buildings 3D" type:Scene Package owner:acrawford_CSOGRD> already on target machine... skipping
Scene Service
detected item <Item title:"Grenada Colored Roof Buildings 3D" type:Scene Layer owner:acrawford_CSOGRD> already on target machine... skipping
Scene Package
detected item <Item title:"Colored Buildings3D" type:Scene Package owner:acrawford_CSOGRD> already on target machine... skipping
Web Map
Shapefile
detected item <Item title:"roads2" type:Shapefile 

_ItemCreateException: ("Failed to create Feature Service Grenada CSO Map: Unable to add feature service definition.\nThe operation failed because an index or statistics with name 'Building_guid_unidx' already exists on table 'db_351.user_351.Grenada_CSO_Map_GRENADA_BUILDING_FOOTPRINTS_211'.\n(Error Code: 400)", <Item title:"Grenada_CSO_Map" type:Feature Layer Collection owner:gtaylor_IVT>)

In [4]:
print("Data in Target User Directory")
target_user_inventory = get_user_items(user=target.users.me, active_gis=target)
print_user_inventory(inventory=target_user_inventory)


Image Service
--------------------------------------------------
   Grenada 2010 Historic Imagery RGB                 
   Grenada 2020 4-band Ortho-Imagery                 
   grenada_elevation                                 


Web Scene
--------------------------------------------------
   Grenada LiDAR                                     
   Grenada Digital Twin from Imagery                 


Web Map
--------------------------------------------------
   Grenada Extracted Buildings                       
   Grenada Imagery                                   
   Building Footprints QA/QC                         


Web Mapping Application
--------------------------------------------------
   Grenada Imagery                                   


Compact Tile Package
--------------------------------------------------
   grenada_elevation                                 


GeoJson
--------------------------------------------------
   WrightsvilleBeach_SiteScan_ImageLocations         
   Wr